In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
capacity = pd.read_csv('https://raw.githubusercontent.com/johnson80245/python/refs/heads/main/csvs/theoretical%20capacity%20data%203%20months.csv')
all_state_aht = pd.read_csv('https://raw.githubusercontent.com/johnson80245/python/refs/heads/main/csvs/AHT_by_state.csv')
def to_clean(val):
    return val.strip().lower().replace(" ","_")

capacity = capacity.rename(columns = to_clean)
#all_state_aht = all_state_aht.rename(columns = to_clean)

In [ ]:
capacity.info()

In [ ]:
#Cast as categorical data
capacity[['userid','position','team_lead_id']] = (capacity[['userid','position','team_lead_id']].astype('category'))
#Cast to float
capacity['adj_total_hours'] = pd.to_numeric(capacity.adj_total_hours, errors = 'coerce')
#Cast to date
capacity['year_month'] = pd.to_datetime(capacity['year_month'])
#Cast to int64
int_columns = ['new_policy_count', 'renewal_count', 'endorsement__count','fte','total_filings','flag_count','srt_reassignments',
               'filing_admin_changes','srt_total','premium_srt','zero_srt','no_charge_srt','ad_hoc_srts']
for column in int_columns:
    capacity[column] = capacity[column].astype('int64')




In [ ]:
#Create AHT based on reported hours
capacity['filing_aht']  = (capacity.filing_hours_self / capacity.total_filings) * 60
capacity['srt_aht']  = (capacity.srt_hours_self / capacity.srt_total) * 60
capacity['flag_aht'] = (capacity['filing_aht'] * 1.25)

#Create Utilization
capacity['filing_utilization'] = (capacity.filing_hours_self/capacity.total_hours_self) 
capacity['srt_utilization'] = (capacity.srt_hours_self/capacity.total_hours_self) 
capacity['total_utilization'] = (capacity.filing_hours_self + capacity.srt_hours_self) / capacity.total_hours_self

In [ ]:
all_state_aht.dtypes

In [ ]:
# Correct way
all_state_aht['transaction_count'] = pd.to_numeric(
    all_state_aht['transaction_count'].astype(str).str.replace(',', ''),
    errors='coerce'
).astype('float64')


In [ ]:
#Create state weighted aht

def compute_weighted_aht(group):
    return (
        (group['transaction_count'] * group['weighted_aht']).sum()
        / group['transaction_count'].sum()
    )

state_trans_aht = (
    all_state_aht
    .groupby(['state', 'transaction_type_group'], group_keys=False)
    .apply(compute_weighted_aht, include_groups=False)
    .reset_index(name='weighted_aht')
)

state_aht = (
    all_state_aht
    .groupby('state', group_keys = False)
    .apply(compute_weighted_aht, include_groups=False)
    .reset_index(name = 'weighted_aht')
)

filing_perc_all = all_state_aht.groupby(['state', 'submission_type'])['transaction_share'].sum().reset_index()
filing_perc_st_group = all_state_aht.groupby(['state','transaction_type_group'])['transaction_share'].sum().reset_index()
filing_perc_st_submission = all_state_aht.groupby(['state', 'submission_type'])['transaction_share'].sum().reset_index()
filing_perc_submission = all_state_aht.groupby(['submission_type'])['transaction_share'].sum().reset_index()
filing_perc_trans_group = all_state_aht.groupby(['transaction_type_group'])['transaction_share'].sum().reset_index()

In [ ]:
filing_perc_st_submission.head()



